In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "Qwen/Qwen3-Embedding-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()


ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Check out the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
from langchain.embeddings.base import Embeddings
from typing import List

class QuenEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._embed(text) for text in texts]

    def embed_query(self, text: str) -> List[float]:
        return self._embed(text)

    def _embed(self, text: str) -> List[float]:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use [CLS] token or mean pooling
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
        return embeddings


In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Example documents
docs = [Document(page_content="anomaly in line 5"), Document(page_content="temperature spike")]

# Initialize vector store
embedding = QuenEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)
